# fast onboarding to add a device to nautobot
With the help of this script you can easily add a device to nautobot

## configure your sot

In [ ]:
token = "your_token"
url = "http://127.0.0.1:8080"
ssl_verify = False

# specify your device

## mandatory settings

In [ ]:
device_name = "test.lab"
primary_ip = "192.168.0.1/24"
primary_interface = "Loopback0"
location = "default-site"
device_type = "iosv"
platform = "ios"

## optional settings
* to add custom_fields use the syntax {'field_name': value}
* to add tags add your tags to the list (eg. tags = ['test'])

In [ ]:
custom_fields = {}
tags = []

## you can overwrite or set any other property of your device

In [ ]:
device_properties = {
    'name': device_name,
    'location': {'name': location},
    'status': {'name': 'Active'},
    'serial': 'unknown',
    'role': {'name': 'network'},
    'device_type': {'model': device_type},
    'platform': {'name': platform},
    'custom_fields': custom_fields,
    'tags': tags
}

interface_properties = [
    {'name': primary_interface,
     'ip_addresses': [{'address': primary_ip,
                       'status': {'name': 'Active'}
                      }],
     'description': 'Primary Interface',
     'type': '1000base-t',
     'status': {'name': 'Active'} 
    }
]
vlan_properties = {}
primary_interface = primary_interface

## set loglevel

In [ ]:
loglevel = "INFO"

# basic setup, import and logging

In [ ]:
import sys
from loguru import logger
from veritas.sot import sot as veritas_sot
from IPython.display import display, Markdown, JSON

## configure logger output

In [ ]:
logger.configure(extra={"extra": "unset"})
logger.remove()
logger.add(sys.stderr, level=loglevel)

## initialize our sot

In [ ]:
sot = veritas_sot.Sot(token=token, url=url, ssl_verify=ssl_verify)

# Onboard device

In [ ]:
already_there = new_device = sot.get.device(name=device_name)
if not already_there:
    new_device = sot.onboarding \
                .interfaces(interface_properties) \
                .vlans(vlan_properties) \
                .primary_interface(primary_interface) \
                .add_prefix(False) \
                .add_device(device_properties)
    if new_device:
        display(f'{new_device.name} added successfully')
    else:
        display(f'onboarding failed')
else:
    display(f'found {device_name} in sot.')

# Further Information
___just needed to get a list of roles, device types and so on___

### show all device types

In [ ]:
display(sot.get.all_device_types(get_list=True))

### show all platforms

In [ ]:
display(sot.get.all_platforms(get_list=True))

### show all locations (list)
* set location type to your need or None to see all
* The output is a condensed list of your locations

In [ ]:
sot.get.all_locations(get_list=True, location_type='ast')

### show all locations (extended output)
* set location type to your need or None to see all
* This output is only useful if your number of locations is low

In [ ]:
import pandas as pd
locations = sot.get.all_locations(get_list=False, location_type=None)
df = pd.DataFrame(locations)
df